Recreate the data we used in the previous Demo

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd

column_names = ['id',
                'clump_thickness',
                'cell_size_uniformity',
                'cell_shape_uniformity',
                'marginal_adhesion',
                'single_epithelial_size',
                'bare_nuclei',
                'bland_chromatin',
                'normal_nucleoli',
                'mitoses',
                'class']

bcw = pd.read_csv('../assets/datasets/breast-cancer-wisconsin.csv',
                 names=column_names, na_values=['?'])

bcw.dropna(inplace=True)
bcw['metrics_pct'] = bcw[[x for x in column_names if x not in ['class','id']]].sum(axis=1)/90.
bcw['class'] = bcw['class'].map(lambda x: 0 if x == 2 else 1)

metrics_pct = np.array(bcw.metrics_pct.values)
metrics_pct = metrics_pct[:, np.newaxis]
X_train, X_test, Y_train, Y_test = train_test_split(metrics_pct, bcw[['class']].values, 
                                                    test_size=0.33, stratify=bcw[['class']].values,
                                                    random_state=77)

logreg = LogisticRegression(random_state=77)
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)

//anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


When we use logreg.predict, what is actually happening here?

The logreg model determines a **predicted probability** of class 0 and class 1 for each observation in the X_test matrix. These probabilities sum to 1 across classes. When predict is called, the standard behavior is to assign the class with the greater probability.

Recreate the Y_pred vector manually from the predicted probability.

In [8]:
# Get the predicted probability vector
Y_pp = pd.DataFrame(logreg.predict_proba(X_test), columns=['class_0_pp','class_1_pp'])
print(Y_pp.iloc[0:10])


   class_0_pp  class_1_pp
0    0.896571    0.103429
1    0.086331    0.913669
2    0.312713    0.687287
3    0.920883    0.079117
4    0.356408    0.643592
5    0.876881    0.123119
6    0.920883    0.079117
7    0.913423    0.086577
8    0.312713    0.687287
9    0.133763    0.866237


This behavior is equivalent to a **classification threshold of 0.5 for class 1**. In other words, if class 1's predicted probability is >= 0.5, the observation is predicted to be class 1.

In [9]:
Y_pp['pred_class_thresh50'] = Y_pred
print(Y_pp.iloc[0:10])

   class_0_pp  class_1_pp  pred_class_thresh50
0    0.896571    0.103429                    0
1    0.086331    0.913669                    1
2    0.312713    0.687287                    1
3    0.920883    0.079117                    0
4    0.356408    0.643592                    1
5    0.876881    0.123119                    0
6    0.920883    0.079117                    0
7    0.913423    0.086577                    0
8    0.312713    0.687287                    1
9    0.133763    0.866237                    1


Let's say again that we are predicting cancer based on some kind of detection measure, as before.

In [10]:
conmat = np.array(confusion_matrix(Y_test, Y_pred, labels=[1,0]))

confusion = pd.DataFrame(conmat, index=['has_cancer', 'is_healthy'],
                         columns=['predicted_cancer','predicted_healthy'])

print(confusion)

            predicted_cancer  predicted_healthy
has_cancer                69                 10
is_healthy                 4                143


In this scenario, we may be particularly interested in minimizing the false positive rate, aka reducing the chance that a patient with cancer is diagnosed as healthy.

In order to do this, we can **lower the threshold for predicting class 1**.

In [11]:
Y_pp['pred_class_thresh10'] = [1 if x >= 0.10 else 0 for x in Y_pp.class_1_pp.values]
print(Y_pp.iloc[0:10])

   class_0_pp  class_1_pp  pred_class_thresh50  pred_class_thresh10
0    0.896571    0.103429                    0                    1
1    0.086331    0.913669                    1                    1
2    0.312713    0.687287                    1                    1
3    0.920883    0.079117                    0                    0
4    0.356408    0.643592                    1                    1
5    0.876881    0.123119                    0                    1
6    0.920883    0.079117                    0                    0
7    0.913423    0.086577                    0                    0
8    0.312713    0.687287                    1                    1
9    0.133763    0.866237                    1                    1


This will reduce our false negative rate to 0, but at the expense of our false positive rate.

In [12]:
conmat_10 = np.array(confusion_matrix(Y_test, Y_pp.pred_class_thresh10.values, labels=[1,0]))

confusion_10 = pd.DataFrame(conmat_10, index=['has_cancer', 'is_healthy'],
                            columns=['predicted_cancer','predicted_healthy'])

print(confusion_10)

            predicted_cancer  predicted_healthy
has_cancer                79                  0
is_healthy                79                 68


A useful way to visualize these tradeoffs is with **area under the curve (AUC)** graphs. The most popular AUC graph is the **reciever operating characteristic (ROC)** curve.

The ROC curve compares the true positive rate against the false positive rate. It is unaffected by the distribution of class labels since it is only comparing the correct vs. incorrect label assignments for one class.

To plot this we will use the roc_curve() and auc() functions from sklearn. We will also use the decision_function() method of the logistic regression, which essentiall gives us the confidence of each observation being in one class or another.

(Plotting is the nearly the same as in the scikit learn documentation)

In [13]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
%matplotlib inline


Y_score = logreg.decision_function(X_test) #predict confidence score for samples


# FPR = dict()  #creates 3 empty dictionaries
# TPR = dict()
# ROC_AUC = dict()

# # For class 1, find the area under the curve
# FPR[1], TPR[1], _ = roc_curve(Y_test, Y_score)
# ROC_AUC[1] = auc(FPR[1], TPR[1])

# # Plot of a ROC curve for class 1 (has_cancer)
# plt.figure(figsize=[11,9])
# plt.plot(FPR[1], TPR[1], label='ROC curve (area = %0.2f)' % ROC_AUC[1], linewidth=4)
# plt.plot([0, 1], [0, 1], 'k--', linewidth=4)
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate', fontsize=18)
# plt.ylabel('True Positive Rate', fontsize=18)
# plt.title('Receiver operating characteristic for cancer detection', fontsize=18)
# plt.legend(loc="lower right")
# plt.show()

[ -2.15969287e+00   2.35928577e+00   7.87467110e-01  -2.45440887e+00
   5.90989778e-01  -1.96321554e+00  -2.45440887e+00  -2.35617020e+00
   7.87467110e-01   1.86809244e+00  -2.35617020e+00   2.16280843e+00
  -2.25793154e+00  -2.35617020e+00  -1.86497687e+00   7.87467110e-01
  -1.27554488e+00   7.87467110e-01  -2.25793154e+00  -1.47202221e+00
   1.27866044e+00  -2.55264754e+00   1.08218311e+00   1.98035113e-01
  -2.15969287e+00  -2.15969287e+00  -1.94919551e-01  -3.91396883e-01
  -2.35617020e+00  -2.55264754e+00   1.55778134e-03  -1.96321554e+00
  -1.96321554e+00  -1.76673821e+00   3.34167243e+00  -2.65088620e+00
   2.96273779e-01   2.16280843e+00   3.34167243e+00   3.24343376e+00
  -2.06145421e+00  -1.94919551e-01   1.67161510e+00   1.55778134e-03
  -2.06145421e+00  -1.96321554e+00  -1.76673821e+00  -2.15969287e+00
   2.26104710e+00  -2.55264754e+00  -2.35617020e+00  -2.35617020e+00
  -2.15969287e+00  -2.15969287e+00  -2.06145421e+00   1.96633110e+00
   2.65400176e+00   2.45752443e+00

What is the intuition for the ROC curve?

As the class assignment threshold increases for the positive class (has cancer), the false positive rate and true positive rate necessarily increase. For a classifier performing at chance, this would be the diagonal dotted line: an equal chance of false positives and true positives. 

The greater the area under the curve, the higher the ratio of true positives to false positives as the threshold becomes more lenient. Thus, the greater the area under the curve, the higher the quality of the classification model. In the Wisconsin breast cancer data the area under the curve is 0.99, indicating a nearly perfect model. Most classification problems will never get close to this!